In [5]:
import requests
import pandas as pd

url = 'https://apis.data.go.kr/1480523/WaterQualityService/getWaterMeasuringList'
key = '8yJUJUA516mCT6EWW5qx4wczXHJvMSZEtuueDB3fhxlXBxkvLuvquPdwtP4CDBkmiHrbiKunquvt/K46xNulRw=='

In [6]:
# 2. 주소에 있던 파라미터 그대로 설정
params = {
    'serviceKey': key,
    'pageNo': '1',
    'numOfRows': '3000',
    'resultType': 'json',
    'ptNoList': '2022A30,2022A10', # 측정소 코드
    'wmyrList': '2021,2022,2023,2024,2025',      # 연도
    'wmodList': '01,02,03,04,05,06,07,08,09,10,11,12'        # 월(01, 02, 03월)
}

try:
    # 3. API 요청
    response = requests.get(url, params=params, verify=True) # https 대응
    
    if response.status_code == 200:
        data = response.json()
        
        # 4. 데이터 추출 (구조: getWaterMeasuringList -> item)
        items = data.get('getWaterMeasuringList', {}).get('item', [])
        
        if items:
            df = pd.DataFrame(items)
            df.set_index('ROWNO',inplace=True)

            # 5. 주요 수질 항목 한글 이름으로 변경 (보기 좋게)
            rename_map = {
                'PT_NM': '총량지점명',
                'WMCYMD': '일자',
                'ITEM_TEMP': '수온(℃)',
                'ITEM_PH': '수소이온농도(ph)',
                'ITEM_EC': '전기전도도(μS/㎝)',
                'ITEM_DOC': '용존산소(㎎/L)',
                'ITEM_BOD': 'BOD(㎎/L)',
                'ITEM_COD': 'COD(㎎/L)',
                'ITEM_SS': '부유물질(㎎/L)',
                'ITEM_TN': '총질소(T-N)(㎎/L)',
                'ITEM_TP': '총인(T-P)(㎎/L)',
                'ITEM_TOC': '총유기탄소(TOC)(㎎/L)',
                'ITEM_AMNT': '유량(㎥/s)',
                'ITEM_CLOA': '클로로필a'
            }
            
            # 2. 필요한 컬럼만 추출하여 새 DF 생성
            waterDF = df[list(rename_map.keys())].copy()
            waterDF.rename(columns=rename_map, inplace=True)

            # 3. 데이터 타입 숫자형으로 변환 (연산 가능하게)
            num_cols = ['수온(℃)', '수소이온농도(ph)', '전기전도도(μS/㎝)', '용존산소(㎎/L)', 'BOD(㎎/L)', 'COD(㎎/L)', 
                        '부유물질(㎎/L)', '총질소(T-N)(㎎/L)', '총인(T-P)(㎎/L)', '총유기탄소(TOC)(㎎/L)', '유량(㎥/s)','클로로필a']
            waterDF[num_cols] = waterDF[num_cols].apply(pd.to_numeric, errors='coerce')

            # # 4. 날짜 데이터 형식 변환 (YYYYMMDD -> datetime)
            waterDF['일자'] = pd.to_datetime(waterDF['일자'], errors='coerce')

            print("--- 분석 준비 완료: 핵심 수질 지표 ---")
            print(waterDF.head())
        else:
            print("응답은 성공했으나 데이터가 없습니다.")
    else:
        print(f"API 요청 실패: {response.status_code}")

except Exception as e:
    print(f"오류 발생: {e}")

--- 분석 준비 완료: 핵심 수질 지표 ---
      총량지점명         일자  수온(℃)  수소이온농도(ph)  전기전도도(μS/㎝)  용존산소(㎎/L)  BOD(㎎/L)  \
ROWNO                                                                         
1        물금 2021-01-04    4.6         7.5        437.0       13.1       1.7   
2        물금 2021-01-11    2.8         7.3        461.0       14.3       2.0   
3        물금 2021-01-26    5.1         7.2        470.0       15.6       2.1   
4        물금 2021-01-18    3.1         7.3        463.0       15.2       2.2   
5        물금 2021-02-16    6.6         8.4        451.0       12.1       2.8   

       COD(㎎/L)  부유물질(㎎/L)  총질소(T-N)(㎎/L)  총인(T-P)(㎎/L)  총유기탄소(TOC)(㎎/L)  \
ROWNO                                                                      
1           5.1        2.4          3.223         0.025              3.8   
2           5.4        4.0          3.268         0.031              4.0   
3           6.0        7.6          3.416         0.026              4.6   
4           5.9        6.4          3.3

In [7]:
waterDF

,총량지점명,일자,수온(℃),수소이온농도(ph),전기전도도(μS/㎝),용존산소(㎎/L),BOD(㎎/L),COD(㎎/L),부유물질(㎎/L),총질소(T-N)(㎎/L),총인(T-P)(㎎/L),총유기탄소(TOC)(㎎/L),유량(㎥/s),클로로필a
ROWNO,,,,,,,,,,,,,,
1,물금,2021-01-04,4.6,7.5,437.0,13.1,1.7,5.1,2.4,3.223,0.025,3.8,29.555,12.4
2,물금,2021-01-11,2.8,7.3,461.0,14.3,2.0,5.4,4.0,3.268,0.031,4.0,28.599,19.4
3,물금,2021-01-26,5.1,7.2,470.0,15.6,2.1,6.0,7.6,3.416,0.026,4.6,74.897,47.3
4,물금,2021-01-18,3.1,7.3,463.0,15.2,2.2,5.9,6.4,3.327,0.032,4.4,47.190,33.2
5,물금,2021-02-16,6.6,8.4,451.0,12.1,2.8,6.6,12.4,3.489,0.027,5.0,72.735,50.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,금곡,2025-09-23,25.4,7.7,285.0,7.8,1.3,6.9,6.4,1.928,0.055,3.6,357.260,17.4
403,금곡,2025-09-09,28.6,8.6,219.0,7.6,1.7,8.6,9.6,1.579,0.059,6.1,340.400,52.6
404,금곡,2025-10-13,23.3,8.4,288.0,9.0,1.6,7.0,7.6,1.858,0.041,3.4,443.810,39.5


In [8]:
waterDF['일자'].dtype

dtype('<M8[ns]')